In [1]:
!pip install ultralytics

import ultralytics
import torch
from IPython.display import Image

ultralytics.checks()






Ultralytics 8.3.142 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.3/235.7 GB disk)


In [2]:
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/ml/training/data.yaml batch=-1 epochs=300 imgsz=1280 plots=True

100% 49.0M/49.0M [00:01<00:00, 44.4MB/s]
Ultralytics 8.3.142 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/ml/training/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mas

In [4]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/drive/MyDrive/ml/training/data.yaml

Ultralytics 8.3.142 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,282,396 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access ✅ (ping: 0.2±0.1 ms, read: 109.9±21.9 MB/s, size: 171.7 KB)
val: Scanning /content/drive/MyDrive/ml/training/valid/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100% 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:04<00:00,  1.02s/it]
                   all         49       1174      0.927      0.877      0.921      0.702
                  ball         45         45      0.877      0.644      0.742      0.385
            goalkeeper         38         39      0.881      0.923      0.971      0.791
                player         49        973      0.986      0.988      0.994       0.86
               referee         49        117      0.965      0.951      0.979      0.773
Speed: 14.2ms preprocess, 14.7m

In [5]:
!zip -r /content/football-model.zip /content/runs/detect/train


  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/train_batch9860.jpg (deflated 4%)
  adding: content/runs/detect/train/train_batch9862.jpg (deflated 4%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 28%)
  adding: content/runs/detect/train/val_batch2_labels.jpg (deflated 2%)
  adding: content/runs/detect/train/args.yaml (deflated 53%)
  adding: content/runs/detect/train/results.png (deflated 8%)
  adding: content/runs/detect/train/F1_curve.png (deflated 9%)
  adding: content/runs/detect/train/val_batch2_pred.jpg (deflated 2%)
  adding: content/runs/detect/train/val_batch0_pred.jpg (deflated 1%)
  adding: content/runs/detect/train/confusion_matrix_normalized.png (deflated 24%)
  adding: content/runs/detect/train/train_batch2.jpg (deflated 1%)
  adding: content/runs/detect/train/val_batch1_labels.jpg (deflated 1%)
  adding: content/runs/detect/train/weights/ (stored 0%)
  adding: content/runs/detect/train/weights/best.pt (deflated 

In [14]:
!pip install opencv-python
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 13.2 MB/s eta 0:00:00


In [34]:
TEST_VIDEO = "/content/test.mp4"
RESULT_VIDEO = "/content/test_result.mp4"

import cv2
from ultralytics import YOLO
import supervision

"""
  Color Order:  1. Ball
                2. Goalie
                3. Player
                4. Ref
"""
annotator = supervision.TriangleAnnotator(
    color = supervision.ColorPalette.from_hex(['#f54254','#42f5d7','#1c3aff','#f5e342']),
    base = 25,
    height = 25
)

yolo_model = YOLO("/content/runs/detect/train/weights/best.pt")
vid = cv2.VideoCapture(TEST_VIDEO)

vid_stats = supervision.VideoInfo.from_video_path(TEST_VIDEO)

sink = supervision.VideoSink(RESULT_VIDEO, video_info=vid_stats)

with sink:

  for frame in supervision.get_video_frames_generator(source_path=TEST_VIDEO):
      print(frame.shape)

  #retval, frame = vid.read()

      result = yolo_model(frame, conf=0.4)[0]

      detections = supervision.Detections.from_ultralytics(result)

      annotated_frame = annotator.annotate(
          scene=frame.copy(),
          detections=detections
      )

      sink.write_frame(annotated_frame)

#supervision.plot_image(annotated_frame)

Streaming output truncated to the last 5000 lines.
(1080, 1920, 3)

0: 736x1280 1 ball, 1 goalkeeper, 13 players, 2 referees, 22.3ms
Speed: 7.4ms preprocess, 22.3ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 1280)
(1080, 1920, 3)

0: 736x1280 1 ball, 1 goalkeeper, 13 players, 2 referees, 19.6ms
Speed: 8.0ms preprocess, 19.6ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 1280)
(1080, 1920, 3)

0: 736x1280 1 ball, 1 goalkeeper, 13 players, 2 referees, 19.5ms
Speed: 7.4ms preprocess, 19.5ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 1280)
(1080, 1920, 3)

0: 736x1280 1 ball, 1 goalkeeper, 14 players, 2 referees, 20.2ms
Speed: 7.8ms preprocess, 20.2ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 1280)
(1080, 1920, 3)

0: 736x1280 1 ball, 1 goalkeeper, 15 players, 2 referees, 21.0ms
Speed: 7.2ms preprocess, 21.0ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)
(1080, 1920, 3)

0: 736x1280 1 ball, 1 goalkeeper